In [ ]:
from collections import defaultdict
import numpy as np
import random
from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px
from copy import copy

<h1> Fish Simulation </h1>

In [ ]:
from matplotlib.colors import Normalize
from matplotlib import cm
from PIL import Image, ImageDraw
import random
import numpy as np
from sympy.solvers import solve
from sympy import Symbol
import uuid
import cv2
sm = cm.ScalarMappable(cmap=cm.get_cmap('Reds'), norm=Normalize(vmin=0.3, vmax=3.0))
from IPython.display import Image as Im


AVG_WEIGHT = 8.0
COV = 0.2
SPEED_FACTOR = 0.7
SPEED_FACTOR_STD = 0.3
MIN_DEPTH = 0.3
MAX_DEPTH = 3.0


def length_from_weight(weight):
    return (weight ** (1 / 3.0)) / 2.36068 * random.gauss(1.0, 0.05)


class Fish:

    def __init__(self, weight_mean, weight_std, speed_factor_mean, speed_factor_std,
                 min_depth, max_depth, max_y_coordinate=3.0):
        self.id = uuid.uuid4()
        self.weight = max(random.gauss(weight_mean, weight_std), 0.1)
        self.length = length_from_weight(self.weight)
        self.height = 0.3 * self.length
        self.depth = random.uniform(min_depth, max_depth)
        self.speed = self.length * random.gauss(speed_factor_mean, speed_factor_std)
        self.is_detected = False
        self.is_occluded = False
        self.position = [-10, self.depth, random.uniform(-max_y_coordinate, max_y_coordinate)]

    def get_position(self):
        return self.position
    
    def update_position(self, delta_t):
        delta_x = self.speed * delta_t
        self.position[0] += delta_x
    
    def get_pixel_bbox(self, camera):
        x_pixel = self.position[0] * camera.focal_length_pixel / self.position[
            1] + camera.pixel_width / 2.0
        y_pixel = -(self.position[2] * camera.focal_length_pixel / self.position[
            1]) + camera.pixel_height / 2.0
        length_pixel = self.length * camera.focal_length_pixel / self.position[1]
        height_pixel = self.height * camera.focal_length_pixel / self.position[1]
        bbox = [x_pixel - length_pixel / 2.0, y_pixel - height_pixel / 2.0,
                x_pixel + length_pixel / 2.0, y_pixel + height_pixel / 2.0]
        return [int(x) for x in bbox]
    
    def get_ellipse_mask(self, camera):
        bbox = self.get_pixel_bbox(camera)
        center = (int(0.5*(bbox[0] + bbox[2])), int(0.5*(bbox[1] + bbox[3])))
        axes = (int(0.5*(bbox[2] - bbox[0])), int(0.5*(bbox[3] - bbox[1])))
        mask = np.zeros([camera.pixel_height, camera.pixel_width])
        mask=cv2.ellipse(mask, center=center, axes=axes, angle=0, startAngle=0, endAngle=360, color=(255,255,255), thickness=-1)
        return mask.astype(int)
        
    


class Camera:

    def __init__(self, position, fov_degrees, aspect_ratio=0.75):
        self.position = position
        self.fov = fov_degrees * np.pi / 180.0
        self.vfov = 2 * np.arctan(np.tan(self.fov / 2) * aspect_ratio)
        self.pixel_width = 1000
        self.pixel_height = int(self.pixel_width * aspect_ratio)
        self.focal_length_pixel = (self.pixel_width / 2) / np.tan(self.fov / 2)


    def contains(self, fish):

        # determine if fish is inside HFOV
        fish_position = fish.get_position()
        fish_segment_at_depth = (fish_position[0] - fish.length / 2.0, fish_position[0] + fish.length / 2.0)
        field_size = 2 * fish_position[1] * np.tan(self.fov / 2.0)
        field_center = self.position[0]
        field_segment_at_depth = (field_center - field_size / 2.0, field_center + field_size / 2.0)
        inside_horizontal_field = (fish_segment_at_depth[0] > field_segment_at_depth[0]) and \
                                  (fish_segment_at_depth[1] < field_segment_at_depth[1])
        
        touches_horizontal_field = ((fish_segment_at_depth[0] < field_segment_at_depth[0]) and \
                                   (fish_segment_at_depth[1] > field_segment_at_depth[0])) or \
                                   ((fish_segment_at_depth[1] > field_segment_at_depth[1]) and \
                                   (fish_segment_at_depth[0] < field_segment_at_depth[1])) or \
                                   inside_horizontal_field

        # determine if fish is inside VFOV
        vertical_fish_segment_at_depth = (
        fish_position[2] - fish.height / 2.0, fish_position[2] + fish.height / 2.0)
        vertical_field_segment_at_depth = (-fish_position[1] * np.tan(self.vfov / 2.0), fish_position[1] * np.tan(self.vfov / 2.0))
        inside_vertical_field = (vertical_fish_segment_at_depth[0] >
                                 vertical_field_segment_at_depth[0]) and \
                                (vertical_fish_segment_at_depth[1] <
                                 vertical_field_segment_at_depth[1])
        
        touches_vertical_field = ((vertical_fish_segment_at_depth[0] < vertical_field_segment_at_depth[0]) and \
                                 (vertical_fish_segment_at_depth[1] > vertical_field_segment_at_depth[0])) or \
                                 ((vertical_fish_segment_at_depth[1] > vertical_field_segment_at_depth[1]) and \
                                 (vertical_fish_segment_at_depth[0] < vertical_field_segment_at_depth[1])) or \
                                 inside_vertical_field

        return inside_horizontal_field and inside_vertical_field, touches_horizontal_field and touches_vertical_field


def is_detected(fish, a=1.5, b=2.5, default_p=1.0):
    return True
    
#     depth = fish.get_position()[1]
#     if depth < a:
#         p = default_p
#     else:
#         p = max(default_p * (b - depth) / (b - a), 0)

#     return random.random() < p


def overlap(fish_1, fish_2, left_camera, right_camera):
    left_overlap = (fish_1.left_ellipse_mask * fish_2.left_ellipse_mask).sum() > 0
    right_overlap = (fish_1.right_ellipse_mask * fish_2.right_ellipse_mask).sum() > 0
    return left_overlap or right_overlap


def get_nonoccluded_fishes(fishes, left_camera, right_camera):
    nonoccluded_fishes = []
    
    net_left_mask = np.zeros((left_camera.pixel_height, left_camera.pixel_width)).astype(int)
    net_right_mask = np.zeros((right_camera.pixel_height, right_camera.pixel_width)).astype(int)
    for idx, fish in enumerate(sorted(fishes, key=lambda x: x.depth)):
        left_mask = fish.get_ellipse_mask(left_camera)
        right_mask = fish.get_ellipse_mask(right_camera)
        left_occlusion = (left_mask * net_left_mask).sum() > 0
        right_occlusion = (right_mask * net_right_mask).sum() > 0
        if not left_occlusion and not right_occlusion:
            nonoccluded_fishes.append(fish)
        net_left_mask = np.bitwise_or(net_left_mask, left_mask)
        net_right_mask = np.bitwise_or(net_right_mask, right_mask)

    return nonoccluded_fishes


def draw_frame(fishes, left_camera):
    im = Image.new('RGB', (left_camera.pixel_width, left_camera.pixel_height))
    draw = ImageDraw.Draw(im)
    for fish in reversed(sorted(fishes, key=lambda x: x.depth)):
        bbox = fish.get_pixel_bbox(left_camera)
        if not fish.is_occluded:
            color = sm.to_rgba(fish.depth, bytes=True)
        else:
            color = (100, 100, 100)
        draw.ellipse(tuple(bbox), fill=color[:3])
        if fish.is_detected:
            draw.rectangle(tuple(bbox), outline=(255, 255, 255), width=2)
    return np.array(im)


def spawn_fish(fishes):
    fish = Fish(AVG_WEIGHT, AVG_WEIGHT * COV, SPEED_FACTOR, SPEED_FACTOR_STD, MIN_DEPTH, MAX_DEPTH)
    fishes.append(fish)


def move_fish(t, t_new, fishes):
    delta_t = t_new - t
    for fish in fishes:
        fish.update_position(delta_t)

    fishes = [fish for fish in fishes if fish.get_position()[0] < 10.0]
    return fishes


def trigger_capture(fishes, sampled_fishes, left_camera, right_camera):
    detected_fishes = []
    fishes_touching_field = []
    for fish in fishes:
        in_left_field, touches_left_field = left_camera.contains(fish)
        in_right_field, touches_right_field = right_camera.contains(fish)
        is_left_detected = in_left_field and is_detected(fish)
        is_right_detected = in_right_field and is_detected(fish)
        if is_left_detected and is_right_detected:
            detected_fishes.append(fish)
        if touches_left_field and touches_right_field:
            fishes_touching_field.append(fish)

    nonoccluded_fishes = get_nonoccluded_fishes(fishes_touching_field, left_camera, right_camera)
    
    detected_fish_ids = [fish.id for fish in detected_fishes]
    touching_field_ids = [fish.id for fish in fishes_touching_field]
    nonoccluded_fish_ids = [fish.id for fish in nonoccluded_fishes]
    
    for fish in fishes:
        fish.is_detected = False
        fish.is_occluded = False
        if fish.id in detected_fish_ids:
            fish.is_detected = True
        if fish.id not in nonoccluded_fish_ids and fish.id in touching_field_ids:
            fish.is_occluded = True
        if fish.is_detected:
            sampled_fishes.append(copy(fish))

    return fishes




In [ ]:
def generate_samples(FOV, FPS, aspect_ratio=0.75):
    fishes = []
    sampled_fishes = []
    left_camera = Camera((0, 0, 0), FOV, aspect_ratio=aspect_ratio)
    right_camera = Camera((0.105, 0, 0), FOV, aspect_ratio=aspect_ratio)

    capture_times = list(np.arange(0, 20000, 1.0 / FPS))
    fish_spawn_times = list(np.cumsum(np.random.exponential(0.1, int(100000))))

    im_arrs = []
    t = 0
    
    while len(capture_times) > 0 and len(fish_spawn_times) > 0:
        event_type = np.argmin([capture_times[0], fish_spawn_times[0]])
        if event_type == 0:
            t_new = capture_times[0]
            fishes = move_fish(t, t_new, fishes)
            fishes = trigger_capture(fishes, sampled_fishes, left_camera, right_camera)
            if 100 < t_new < 200:
                im_arr = draw_frame(fishes, left_camera)
                im_arrs.append(im_arr)
            t = t_new
            del capture_times[0]
            
            if len(capture_times) % 10 == 0:
                print(len(capture_times))
            
        elif event_type == 1:
            t_new = fish_spawn_times[0]
            fishes = move_fish(t, t_new, fishes)
            spawn_fish(fishes)
            t = t_new
            del fish_spawn_times[0]

    return sampled_fishes, im_arrs


In [ ]:
samples, im_arrs = generate_samples(54, 0.6)

In [ ]:
nonoccluded_fishes = [fish for fish in samples if not fish.is_occluded]

In [ ]:
unique_fish_dict = {}
for idx, fish in enumerate(samples):
    fish_id = str(fish.id)
    if fish_id not in unique_fish_dict:
        unique_fish_dict[fish_id] = fish

unique_fishes = list(unique_fish_dict.values())

unique_nonoccluded_fish_dict = {}
for fish in nonoccluded_fishes:
    fish_id = str(fish.id)
    if fish_id not in unique_nonoccluded_fish_dict:
        unique_nonoccluded_fish_dict[fish_id] = fish


unique_nonoccluded_fishes = list(unique_nonoccluded_fish_dict.values())

In [ ]:
np.mean([fish.weight for fish in unique_fishes if 0.0 < fish.depth < 5])

In [ ]:
np.mean([fish.weight for fish in unique_nonoccluded_fishes if 0.0 < fish.depth < 5])

In [ ]:
depth_cutoffs = list(np.arange(0.5, 2.5, 0.1))
depth_buckets, sample_sizes, pct_errors = [], [], []
for low_d, high_d in zip(depth_cutoffs, depth_cutoffs[1:]):
    depth_bucket = '{}-{}'.format(round(low_d, 1), round(high_d, 1))
    fish_subset = [fish for fish in unique_fishes if low_d < fish.depth < high_d]
    sample_size = len(fish_subset)
    avg_weight = np.mean([fish.weight for fish in fish_subset])
    pct_error = (avg_weight - 8) / 8

    depth_buckets.append(depth_bucket)
    sample_sizes.append(sample_size)
    pct_errors.append(pct_error)

analysis_df = pd.DataFrame({
    'depth_bucket': depth_buckets,
    'sample_size': sample_sizes,
    'pct_error': pct_errors
})

fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size', color='red')
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()
ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error', color='blue')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
depth_cutoffs = list(np.arange(0.5, 2.5, 0.1))
depth_buckets, sample_sizes, pct_errors = [], [], []
for low_d, high_d in zip(depth_cutoffs, depth_cutoffs[1:]):
    depth_bucket = '{}-{}'.format(round(low_d, 1), round(high_d, 1))
    fish_subset = [fish for fish in unique_nonoccluded_fishes if low_d < fish.depth < high_d]
    sample_size = len(fish_subset)
    avg_weight = np.mean([fish.weight for fish in fish_subset])
    pct_error = (avg_weight - 8) / 8

    depth_buckets.append(depth_bucket)
    sample_sizes.append(sample_size)
    pct_errors.append(pct_error)

analysis_df_2 = pd.DataFrame({
    'depth_bucket': depth_buckets,
    'sample_size': sample_sizes,
    'pct_error': pct_errors
})

fig, ax = plt.subplots(figsize=(20, 10))
ax.bar(analysis_df.depth_bucket, analysis_df.sample_size, label='sample size - fish are see through', color='blue', alpha=0.5)
ax.bar(analysis_df_2.depth_bucket, analysis_df_2.sample_size, label='sample size - occluded fish removed', color='red', alpha=0.5)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel('Distance-from-camera bucket (m)')

ax2=ax.twinx()

ax2.plot(analysis_df.depth_bucket, analysis_df.pct_error, label='pct error - fish are see through', color='green')
ax2.plot(analysis_df_2.depth_bucket, analysis_df_2.pct_error, label='pct error - occluded fish removed', color='red')
ax.grid()
ax.legend(loc='lower right')
ax2.legend(loc='upper right')
ax2.axhspan(-0.02, 0.02, color='red', alpha=0.3)

ax.set_ylabel('Sample Size')
ax2.set_ylabel('Pct. error')
plt.show()

In [ ]:
fs = []
for idx, im_arr in enumerate(im_arrs):
    im = Image.fromarray(im_arr)
    f = '/root/data/alok/biomass_estimation/playground/fov_simulation/im_{}.jpg'.format(idx)
    im.save(f)
    fs.append(f)

In [ ]:
[f.replace('/root', '') for f in fs]

In [ ]:
stitch_frames_into_video(image_fs, '/data/alok/biomass_estimation/playground/fov_simulation.avi')